In [1]:
from __future__ import print_function, division

import os
import time
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# pytorch imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.autograd import Variable
from torch.optim import lr_scheduler
from ignite.metrics import EpochMetric
from ignite.contrib.metrics import roc_auc, ROC_AUC, RocCurve
from torchvision import datasets, transforms
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.engine import create_supervised_evaluator, create_supervised_trainer, Events
from ignite.metrics import Accuracy, Precision, Recall, ConfusionMatrix, Fbeta, Loss, MetricsLambda

##### Functions to help set the final classifier layer and other basic model features

In [2]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [3]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    # variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

##### Pre-process the data to make sure it is properly formatted for training and evaluation

In [4]:
# pre-process the image data for training and validation sets

def preprocess_data(train_data_dir = '/Users/jacksimonson/Documents/CBIS-DDSM Train',
                    test_data_dir = '/Users/jacksimonson/Documents/CBIS-DDSM Val'):

    # transform the train and validation data
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize([224, 224]),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # standard pytorch normalization values
        ]),
        'val': transforms.Compose([
            transforms.Resize([224, 224]),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }


    # collect in a dictionary
    image_datasets = {}
    image_datasets['train'] = datasets.ImageFolder(train_data_dir, data_transforms['train'])
    image_datasets['val'] = datasets.ImageFolder(test_data_dir, data_transforms['val'])


    # create data loaders
    dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
    class_names = image_datasets['val'].classes

    return class_names, dataset_sizes, dataloaders_dict

##### This function is the global one. It initializes the model, the parameters that need to be trained/tuned, processes the data, and runs the trainier and executor.

In [5]:
def build_and_train(model_name = 'alexnet', num_classes = 2, num_epochs = 10, device = torch.device("cpu"),
                    feature_extract = True, criterion = nn.CrossEntropyLoss()):
    # Initialize the model for this run
    model = [models.resnet18(pretrained=use_pretrained) if model_name == 'resnet' else models.resnet18(pretrained=use_pretrained)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    input_size = 224
    model, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

    # Print the model we just instantiated
    print(model)

    # get data downloaders
    class_names, dataset_sizes, dataloaders_dict = preprocess_data()
    
    # Send the model to GPU/CPU
    model = model.to(device)

    # Gather the parameters to be optimized/updated in this run
    params_to_update = model.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                print("\t",name)

    # Observe that all parameters are being optimized
    optimizer_ft = optim.Adam(params_to_update, lr=0.001)
    
    train_dl = dataloaders_dict['train']
    val_dl = dataloaders_dict['val']
    
    
    # create a trainer and evaluator using the ignite package to train and evaluate the models
    trainer = create_supervised_trainer(model, optimizer_ft, criterion, device)
    evaluator = create_supervised_evaluator(model,
                                            metrics={'accuracy': Accuracy(),
                                                     'confusion': ConfusionMatrix(num_classes = 2),
                                                     'loss': Loss(criterion)},
                                            device=device)
    
    # attach additional metrics
    precision = Precision(average=False)
    recall = Recall(average=False)
    F1 = (precision * recall * 2 / (precision + recall)).mean()

    precision.attach(evaluator, 'precision')
    recall.attach(evaluator, 'recall')
    F1.attach(evaluator, 'F1')
    
    
    @trainer.on(Events.EPOCH_COMPLETED)
    def log_training_results(trainer):
        evaluator.run(train_dl)
        metrics = evaluator.state.metrics
#         pred, y = evaluator.state.output
#         ra = ROC_AUC(activated_output_transform(evaluator.state.output), True)
#         val = ra.roc_auc_compute_fn([pred, y])
        print(
            f"Training Results   - Epoch: {trainer.state.epoch}  "
            f"accuracy: {metrics['accuracy']:.2f} "
            f"loss: {metrics['loss']:.2f} "
            f"prec: {metrics['precision'].cpu()} "
            f"recall: {metrics['recall'].cpu()} "
            f"F1: {metrics['F1']}")
#             f"ROC AUC: {roc_auc_val:.2f} "
#             f"ROC Curve: {roc_auc_curve:.2f}"
#         )


    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(engine):
        evaluator.run(val_dl)
        metrics = evaluator.state.metrics
#         pred, y = evaluator.state.output
        print(
            f"Validation Results - Epoch: {trainer.state.epoch}  "
            f"accuracy: {metrics['accuracy']:.2f} "
            f"loss: {metrics['loss']:.2f} "
            f"prec: {metrics['precision'].cpu()} "
            f"recall: {metrics['recall'].cpu()} "
            f"F1: {metrics['F1']}")
#             f"ROC AUC: {roc_auc_val:.2f} "
#             f"ROC Curve: {roc_auc_curve:.2f}"
#         )


    trainer.run(train_dl, max_epochs=15)
    return trainer, evaluator

In [6]:
# run the analysis on alexnet and resnet
models_final = {}
for model in ['alexnet', 'resnet']:
    trainer, evaluator = build_and_train(model)
    models_final[model] = (trainer, evaluator)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

Training Results   - Epoch: 1  accuracy: 0.55 loss: 0.86 prec: tensor([0.9024, 0.5429], dtype=torch.float64) recall: tensor([0.0470, 0.9955], dtype=torch.float64) F1: 0.3959515039449723
Validation Results - Epoch: 1  accuracy: 0.49 loss: 0.99 prec: tensor([0.9231, 0.4735], dtype=torch.float64) recall: tensor([0.0480, 0.9953], dtype=torch.float64) F1: 0.3664669566357506
Training Results   - Epoch: 2  accuracy: 0.67 loss: 0.61 prec: tensor([0.7568, 0.6342], dtype=torch.float64) recall: tensor([0.4226, 0.8806], dtype=torch.float64) F1: 0.6398642272215289
Validation Results - Epoch: 2  accuracy: 0.59 loss: 0.69 prec: tensor([0.6928, 0.5385], dtype=torch.float64) recall: tensor([0.4240, 0.7814], dtype=torch.float64) F1: 0.5818128740329879
Training Results   - Epoch: 3  accuracy: 0.69 loss: 0.58 prec: tensor([0.7390, 0.6697], dtype=torch.float64) recall: tensor([0.5317, 0.8348], dtype=torch.float64) F1: 0.6808097917044744
Validation Results - Epoch: 3  accuracy: 0.62 loss: 0.66 prec: tensor(